In [36]:
library('RODBC')
library('quantmod')
library('PerformanceAnalytics')
source("/usr/common/config.R")
lcon <- odbcDriverConnect(sprintf("Driver={SQL Server};Server=%s;Database=%s;Uid=%s;Pwd=%s;", ldbserver, ldbname, ldbuser, ldbpassword), case = "nochange", believeNRows = TRUE)


In [42]:
dates<-sqlQuery(lcon, "select distinct(time_stamp) from EQUITY_MISC_INFO where LOWER_PX_BAND is not null")

print(sprintf("%s through %s (n=%d)", first(dates[,1]), last(dates[,1]), length(dates[,1])))

ulCkt<-data.frame(DECILE=0, SYMBOL="", TIME_STAMP=as.Date('2000-01-01'), IS_LT=F, IS_UT=F, stringsAsFactors = F)
for(i in 1:length(dates[,1])){
    d<-toString(dates[i,1])

    ckt<-sqlQuery(lcon, sprintf("select DECILE, m.SYMBOL, m.TIME_STAMP, case when p.PX_LOW <= m.LOWER_PX_BAND then 1 else 0 end as IS_LT, case when p.PX_HIGH >= m.UPPER_PX_BAND then 1 else 0 end as IS_UT
                                    from EQUITY_MISC_INFO m, PX_HISTORY p, DECILE_CONSTITUENTS d
                                    where m.SYMBOL = p.SYMBOL
                                    and m.SYMBOL = d.SYMBOL
                                    and m.TIME_STAMP = p.TIME_STAMP
                                    and d.TIME_STAMP = (select max(time_stamp) from DECILE_CONSTITUENTS d2 where d2.TIME_STAMP <= p.TIME_STAMP)
                                    and (p.PX_LOW <= m.LOWER_PX_BAND or p.PX_HIGH >= m.UPPER_PX_BAND)
                                    and p.TIME_STAMP = '%s'
                                    and (p.SERIES = 'eq' or p.SERIES='be')
                                    and p.PX_CLOSE > 10", d))
    
    if(length(ckt[,1]) == 0) next
    ckt[,3]<-as.Date(ckt[,3])
    ulCkt<-rbind(ulCkt, ckt)
}
    
ulCkt<-ulCkt[-1,]

print(head(ulCkt))
print(tail(ulCkt))

[1] "2017-05-10 through 2017-06-08 (n=25)"
  DECILE     SYMBOL TIME_STAMP IS_LT IS_UT
2      8 JENSONICOL 2017-05-10     1     0
3      8     KERNEX 2017-05-10     1     0
4      7 MADRASFERT 2017-05-10     0     1
5      7  MURUDCERA 2017-05-10     0     1
6      3     SHILPI 2017-05-10     1     0
7      6  SPMLINFRA 2017-05-10     0     1
     DECILE     SYMBOL TIME_STAMP IS_LT IS_UT
1328      9      KIOCL 2017-06-08     0     1
1329      6   MBLINFRA 2017-06-08     0     1
1330      4     SHILPI 2017-06-08     1     0
1331      9        TFL 2017-06-08     0     1
1332      7 UBHOLDINGS 2017-06-08     1     0
1333      9   VIMALOIL 2017-06-08     1     0


In [38]:
lowerCktCount1<-aggregate(IS_LT~SYMBOL, data=ulCkt, FUN=sum)
lowerCktCount2<-aggregate(DECILE~SYMBOL, data=ulCkt, FUN=mean)
lowerCktCount<-merge(lowerCktCount1, lowerCktCount2, by='SYMBOL')

upperCktCount1<-aggregate(IS_UT~SYMBOL, data=ulCkt, FUN=sum)
upperCktCount2<-aggregate(DECILE~SYMBOL, data=ulCkt, FUN=mean)
upperCktCount<-merge(upperCktCount1, upperCktCount2, by='SYMBOL')

In [39]:
lowerCktCount<-lowerCktCount[order(lowerCktCount[,2], decreasing=T),]
print(lowerCktCount[lowerCktCount[,2] > length(dates[,1])/2 & lowerCktCount[,3] <= 8,])

        SYMBOL IS_LT   DECILE
183     SHILPI    24 3.541667
23   BEARDSELL    16 8.000000
225   VIDEOIND    16 2.000000
65   GOLDINFRA    14 6.000000
48  DSKULKARNI    13 7.000000


In [40]:
upperCktCount<-upperCktCount[order(upperCktCount[,2], decreasing=T),]
print(upperCktCount[upperCktCount[,2] > length(dates[,1])/2 & upperCktCount[,3] <= 8,])

   SYMBOL IS_UT DECILE
34  CANDC    14      7
